In [13]:
#@title import libraries

import numpy as np
import tensorflow as tf

from tensorflow.keras.datasets import fashion_mnist

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical

from tensorflow.keras import backend as K

In [2]:
#@title load data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [3]:
#@title normalize data
x_train = x_train / x_train.max()
x_test = x_test / x_test.max()

# check the range
x_train.max()

1.0

In [4]:
#@title reshape data for MLP
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# check the shape
x_train.shape

(60000, 784)

In [5]:
#@title one hot encode target
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# check shape
y_train.shape

(60000, 10)

In [6]:
#@title create simple model
model = Sequential()
model.add(Input(shape=(784,)))
model.add(Dense(500, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               392500    
                                                                 
 dense_1 (Dense)             (None, 500)               250500    
                                                                 
 dense_2 (Dense)             (None, 256)               128256    
                                                                 
 dense_3 (Dense)             (None, 10)                2570      
                                                                 
Total params: 773,826
Trainable params: 773,826
Non-trainable params: 0
_________________________________________________________________


In [7]:
K.cast

<function keras.backend.cast>

In [8]:
def kl_div(y_true, y_pred):

    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    y_true = K.clip(y_true, K.epsilon(), 1)
    y_pred = K.clip(y_pred, K.epsilon(), 1)

    return tf.reduce_sum(y_true * tf.math.log(y_true / y_pred), axis=-1)

In [9]:
# compile model with loss as KL divergence
model.compile(loss=kl_div,
              optimizer="adam",
              metrics=['accuracy'])

In [10]:
#@title train model
model.fit(x_train, y_train,batch_size=256,epochs=10)

Epoch 1/10
235/235 [==============================] - 2s 6ms/step - loss: 0.5089 - accuracy: 0.8183
Epoch 2/10
235/235 [==============================] - 1s 6ms/step - loss: 0.3574 - accuracy: 0.8695
Epoch 3/10
235/235 [==============================] - 1s 6ms/step - loss: 0.3129 - accuracy: 0.8861
Epoch 4/10
235/235 [==============================] - 1s 6ms/step - loss: 0.2939 - accuracy: 0.8910
Epoch 5/10
235/235 [==============================] - 1s 6ms/step - loss: 0.2730 - accuracy: 0.8977
Epoch 6/10
235/235 [==============================] - 1s 6ms/step - loss: 0.2613 - accuracy: 0.9012
Epoch 7/10
235/235 [==============================] - 1s 6ms/step - loss: 0.2480 - accuracy: 0.9061
Epoch 8/10
235/235 [==============================] - 1s 5ms/step - loss: 0.2304 - accuracy: 0.9135
Epoch 9/10
235/235 [==============================] - 1s 6ms/step - loss: 0.2207 - accuracy: 0.9167
Epoch 10/10
235/235 [==============================] - 1s 6ms/step - loss: 0.2122 - accuracy: 0.9194

In [12]:
#@title evaluate on validation data
score = model.evaluate(x_test, y_test)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

313/313 [==============================] - 1s 3ms/step - loss: 0.3336 - accuracy: 0.8900
Test loss: 0.3335549533367157 / Test accuracy: 0.8899999856948853
